# Suite database viewer
This notebook was created to capture common commands which may be useful to developers
working on, or helping users with Cylc Suite databases.


In [105]:
# Set the parameters of the suite whose database you want to investigate
USER = "jbloggs"
SUITENAME = "simplest"
SUITE_FILE_GLOB_PATTERN = f"/home/*/{USER}/cylc-run/{SUITENAME}/"

# Deliberately left blank before sharing
SQLLITEPATH = "/home/h02/tpilling/miniconda3/bin/sqlite3"

In [106]:
# This cell contains boilerplate to ensure that the database exists
from IPython.core.display import HTML, display, Markdown
from glob import glob

def hide_cell(comment=''):
    # Taken from https://stackoverflow.com/questions/31517194/
    # how-to-hide-one-specific-cell-input-or-output-in-ipython-notebook
    injecttxt = """

<script>
code_show=true;
function code_toggle() {
    if (code_show){
        $('div.cell.code_cell.rendered.selected div.input').hide();
    } else {
      $('div.cell.code_cell.rendered.selected div.input').show();
    }
    code_show = !code_show
}
$( document ).ready(code_toggle);
</script>
To show/hide this cell's raw code input, click
<a href="javascript:code_toggle()">here</a>.
"""
    injecttxt = comment + injecttxt + "\n\n---"

    printm(injecttxt)
    
def printm(text):
    display(Markdown(text))

hide_cell(
    """---
**NOTE**

This cell contains boilerplate that mostly you don't need to see.
It checks that the information you have given returns just 1 database file.
"""
)
    
SUITE_FOLDER=glob(SUITE_FILE_GLOB_PATTERN)
if len(SUITE_FOLDER) == 1:
    dbfile=f"{SUITE_FOLDER[0]}log/db"
    printm("This is the datebase file we shall interrogate:")
    !ls -l $dbfile
else:
    msg = (
        f"\nthis username and suite name do not produce a unique answer."
        f"\nThe following files were found:\n"
        f"{SUITE_FOLDER}"
    )
    raise FileNotFoundError(msg)


---
**NOTE**

This cell contains boilerplate that mostly you don't need to see.
It checks that the information you have given returns just 1 database file.


<script>
code_show=true;
function code_toggle() {
    if (code_show){
        $('div.cell.code_cell.rendered.selected div.input').hide();
    } else {
      $('div.cell.code_cell.rendered.selected div.input').show();
    }
    code_show = !code_show
}
$( document ).ready(code_toggle);
</script>
To show/hide this cell's raw code input, click
<a href="javascript:code_toggle()">here</a>.


---

FileNotFoundError: 
this username and suite name do not produce a unique answer.
The following files were found:
[]

## SQL introspection
### Tables
Firstly we can get a list of tables available...

In [95]:
%%script $SQLLITEPATH $dbfile

. tables

broadcast_events              task_events                 
broadcast_states              task_jobs                   
broadcast_states_checkpoints  task_late_flags             
checkpoint_id                 task_outputs                
inheritance                   task_pool                   
suite_params                  task_pool_checkpoints       
suite_params_checkpoints      task_states                 
suite_template_vars           task_timeout_timers         
task_action_timers            xtriggers                   


### Schema
A good deal more info comes from the schema...

In [96]:
%%capture N
%%script $SQLLITEPATH  $dbfile

. schema

In [97]:
for item in N.stdout.split('\n'):
    print(item, '\n')

CREATE TABLE suite_params(key TEXT, value TEXT, PRIMARY KEY(key)); 

CREATE TABLE task_jobs(cycle TEXT, name TEXT, submit_num INTEGER, is_manual_submit INTEGER, try_num INTEGER, time_submit TEXT, time_submit_exit TEXT, submit_status INTEGER, time_run TEXT, time_run_exit TEXT, run_signal TEXT, run_status INTEGER, user_at_host TEXT, batch_sys_name TEXT, batch_sys_job_id TEXT, PRIMARY KEY(cycle, name, submit_num)); 

CREATE TABLE task_late_flags(cycle TEXT, name TEXT, value INTEGER, PRIMARY KEY(cycle, name)); 

CREATE TABLE broadcast_states_checkpoints(id INTEGER, point TEXT, namespace TEXT, key TEXT, value TEXT, PRIMARY KEY(id, point, namespace, key)); 

CREATE TABLE checkpoint_id(id INTEGER, time TEXT, event TEXT, PRIMARY KEY(id)); 

CREATE TABLE inheritance(namespace TEXT, inheritance TEXT, PRIMARY KEY(namespace)); 

CREATE TABLE suite_params_checkpoints(id INTEGER, key TEXT, value TEXT, PRIMARY KEY(id, key)); 

CREATE TABLE task_pool_checkpoints(id INTEGER, cycle TEXT, name TEXT, spaw

## Diving into the data
We should now have enough information about the data to start looking in detail at what the database contains:

In [107]:
%%script $SQLLITEPATH $dbfile

SELECT cycle, name,user_at_host, batch_sys_name from task_jobs 
LIMIT 10

19831213T0600Z|Quokka|vld398.cmpd1.metoffice.gov.uk|background
19831213T0600Z|Numbat|vld398.cmpd1.metoffice.gov.uk|background
19831213T0600Z|Wambenger|vld398.cmpd1.metoffice.gov.uk|background
19831213T0700Z|Quokka|vld398.cmpd1.metoffice.gov.uk|background
19831213T0700Z|Numbat|vld398.cmpd1.metoffice.gov.uk|background
19831213T0700Z|Wambenger|vld398.cmpd1.metoffice.gov.uk|background
19831213T0800Z|Quokka|vld398.cmpd1.metoffice.gov.uk|background
19831213T0800Z|Numbat|vld398.cmpd1.metoffice.gov.uk|background
19831213T0800Z|Wambenger|vld398.cmpd1.metoffice.gov.uk|background
19831213T0900Z|Quokka|vld398.cmpd1.metoffice.gov.uk|background


... or, if you want to be really fancy this sort of thing will give you a nicely formatted pandas
table

In [102]:
%%capture M
%%script $SQLLITEPATH $dbfile

SELECT cycle, name, user_at_host, batch_sys_name from task_jobs 
LIMIT 10

In [103]:
from io import StringIO
pd.read_csv(
    StringIO(M.stdout),
    sep="|",
    index_col=False,
    names=[
        "cycle", 
        "name", 
        "user_at_host", 
        "batch_sys_name"
    ]
)

,cycle,name,user_at_host,batch_sys_name
0,19831213T0600Z,Quokka,vld398.cmpd1.metoffice.gov.uk,background
1,19831213T0600Z,Numbat,vld398.cmpd1.metoffice.gov.uk,background
2,19831213T0600Z,Wambenger,vld398.cmpd1.metoffice.gov.uk,background
3,19831213T0700Z,Quokka,vld398.cmpd1.metoffice.gov.uk,background
4,19831213T0700Z,Numbat,vld398.cmpd1.metoffice.gov.uk,background
5,19831213T0700Z,Wambenger,vld398.cmpd1.metoffice.gov.uk,background
6,19831213T0800Z,Quokka,vld398.cmpd1.metoffice.gov.uk,background
7,19831213T0800Z,Numbat,vld398.cmpd1.metoffice.gov.uk,background
8,19831213T0800Z,Wambenger,vld398.cmpd1.metoffice.gov.uk,background
9,19831213T0900Z,Quokka,vld398.cmpd1.metoffice.gov.uk,background
